In [2]:
import torch
from torch import nn


In [3]:
def corr_2d(X,k):
    h,w=k.shape
    y=torch.zeros(X.shape[0]-h+1,X.shape[1]-w+1)
    for i in range(y.shape[0]):
        for j in range(y.shape[1]):
            y[i][j]=(X[i:i+h,j:j+w]*k).sum()
    return y

In [4]:
X=torch.randint(1, 10, (100,100))
k=torch.randint(1, 10, (10,10))
corr_2d(X,k)



tensor([[2579., 2622., 2649.,  ..., 2350., 2309., 2401.],
        [2670., 2596., 2604.,  ..., 2315., 2490., 2368.],
        [2716., 2720., 2725.,  ..., 2217., 2381., 2171.],
        ...,
        [2561., 2455., 2392.,  ..., 2402., 2429., 2623.],
        [2536., 2432., 2471.,  ..., 2382., 2391., 2384.],
        [2566., 2465., 2442.,  ..., 2505., 2442., 2556.]])

In [34]:
class Conv2D(nn.Module):
    def __init__(self,k_size):
        super().__init__()
        self.weight=nn.Parameter(torch.rand(k_size[0],k_size[1]))
        self.bias=nn.Parameter(torch.zeros(1))
    def forward(self,input):
        return corr_2d(input,self.weight)+self.bias

In [35]:
val=(k.shape[0],k.shape[1])
conv=Conv2D(val)

In [36]:
conv(X)

tensor([[281.7223, 268.7761, 266.1966,  ..., 250.0331, 246.1008, 238.8815],
        [270.3250, 263.3986, 257.1867,  ..., 243.8959, 252.3778, 228.7650],
        [284.6339, 264.2975, 262.6812,  ..., 232.8015, 244.4619, 229.3676],
        ...,
        [250.5531, 230.8836, 239.7252,  ..., 254.6026, 245.5952, 240.9799],
        [256.0441, 243.6381, 243.2832,  ..., 248.1702, 252.3488, 258.0821],
        [267.5214, 258.7990, 254.3589,  ..., 249.0653, 233.3763, 249.0046]],
       grad_fn=<AddBackward0>)

#object detection

Let us take a moment to parse a simple application of a convolutional layer: detecting the edge ofan object in an image by finding the location of the pixel change. First, we construct an “image”of68pixels. The middle four columns are black (0) and the rest are white (1)

Next, we construct a kernelKwith a height of 1 and a width of 2. When we perform the cross-correlation operation with the input, if the horizontally adjacent elements are the same, the outputis 0. Otherwise, the output is non-zero.

In [43]:
X=torch.ones(6,8)
k=torch.tensor([[1.0,-1.0]])
X[:,2:6]=0
y=corr_2d(X,k)

In [44]:
y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [46]:
y.shape

torch.Size([6, 7])

tensor(0.)

In [58]:
#learning a kernel
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
X=X.reshape(1,1,6,8)
y=y.reshape(1,1,6,7)
lr=3e-2
optimizer=torch.optim.SGD(conv2d.parameters(),lr=lr)
loss=nn.MSELoss()
for i in range(10):
    y_hat=conv2d(X)
    l=(y_hat-y)**2
    optimizer.zero_grad()
    l.sum().backward()
    optimizer.step()
    if (i+1)%2==0:
        print(f'Iteration {i+1} , loss {l.sum():.3f}')






Iteration 2 , loss 15.810
Iteration 4 , loss 4.729
Iteration 6 , loss 1.644
Iteration 8 , loss 0.624
Iteration 10 , loss 0.247


In [59]:
conv2d.weight.data.reshape((1,2))

tensor([[ 1.0368, -0.9352]])

The convolution operation is very similar to the cross-correlation operation but has a slight difference. In Convolution operation, the kernel is first flipped by an angle of 180 degrees and is then applied to the image. The fundamental property of convolution is that convolving a kernel with a discrete unit impulse yields a copy of the kernel at the location of the impulse.




The Basic difference between Correlation and convolution is :-
Correlation is measurement of the similarity between two signals/sequences.
Convolution is measurement of effect of one signal on the other signal.

In [39]:
X

tensor([[[[0.1767, 0.9331, 0.1084, 0.9520, 0.7184, 0.4365, 0.1569, 0.8390],
          [0.8747, 0.1435, 0.1686, 0.4194, 0.6186, 0.1667, 0.3601, 0.7520],
          [0.0444, 0.4910, 0.9677, 0.9051, 0.0122, 0.4588, 0.6456, 0.4575],
          [0.8757, 0.4201, 0.5241, 0.4255, 0.5034, 0.2647, 0.8301, 0.1406],
          [0.6907, 0.5136, 0.0228, 0.5339, 0.7769, 0.7910, 0.0781, 0.5106],
          [0.3486, 0.4837, 0.6540, 0.5129, 0.2657, 0.7830, 0.0229, 0.8276]]]])

1

In [27]:
val[length//2:]

'39'

In [118]:
arr=[3,1,2,4]
t=7
arr=sorted(arr)
triplets=[]
for i in range(len(arr)):
    j=i+1
    k=len(arr)-1
    while j<k and arr[i]+arr[j]<t:
        if arr[i]+arr[j]+arr[k]<=t:
            triplets+=[[arr[i],arr[j],arr[val]] for val in range(j+1,k+1)]
            triplets
            j+=1
        else:
            k-=1

In [116]:
len(triplets)

2

In [94]:
[1,2,3,4,5]
t=8

In [99]:
triplets

[[1, 2, 5], [1, 3, 4]]